# Creazione matrici per ogni sottopercorso in base alla mossa

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

path = "DB-Output_original.csv"
data = pd.read_csv(path)

#data = data[:9000] # posso ridurre da 35mila a 9mila senza perdere i luoghi, velocizzando l'esecuzione
data = data.loc[:, ['Initial Solution', 'Moves']]

data.head(5)

,Initial Solution,Moves
0,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'DemandBasedDestroyCustomer',..."
1,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'ShawDestroyCustomer', 'Greed..."
2,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'WorstTimeDestroyCustomer', '..."
3,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'GreedyRouteRemoval', 'Greedy..."
4,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['RandomDestroyStation', 'ProbabilisticBestRep..."


# Lista delle mosse disponibili e applicate

A partire dal dataframe iniziale, si crea una lista chiamata "mv", dove al suo interno sono presenti tutte le diverse mosse applicate e riportate nella tabella.

In [2]:
mv = data["Moves"]
moves = []
for row in mv:
    for elem in row.split("', '"):
        moves.append(elem)

for i in range(len(moves)):
    moves[i] = moves[i].replace("['","")
    moves[i] = moves[i].replace("']","")

to_remove = "null"
moves = list(filter(lambda x: x != to_remove, moves))

moves=list(set(moves))

print("Lista delle", len(moves), "possibili e diverse mosse: \n", moves)

Lista delle 16 possibili e diverse mosse: 
 ['ShawDestroyCustomer', 'GreedyDestroyCustomer', 'DeterministicBestRepairStation', 'LongestWaitingTimeDestroyStation', 'WorstTimeDestroyCustomer', 'GreedyRouteRemoval', 'ProbabilisticWorstRemovalCustomer', 'WorstDistanceDestroyCustomer', 'ProximityBasedDestroyCustomer', 'DemandBasedDestroyCustomer', 'GreedyRepairCustomer', 'ZoneDestroyCustomer', 'RandomRouteDestroyCustomer', 'RandomDestroyStation', 'TimeBasedDestroyCustomer', 'ProbabilisticBestRepairStation']


In [3]:
data["Moves"] = data["Moves"].str.replace("[","")
data["Moves"] = data["Moves"].str.replace("]","")
data["Moves"] = data["Moves"].str.replace("'","")
data.head(5)

,Initial Solution,Moves
0,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, DemandBasedDestroyCustomer, Greedy..."
1,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, ShawDestroyCustomer, GreedyRepairC..."
2,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, WorstTimeDestroyCustomer, GreedyRe..."
3,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
4,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","RandomDestroyStation, ProbabilisticBestRepairS..."


In [4]:
user_input = input("Inserisci una stringa: ")
data = data.loc[data['Moves'].str.contains(user_input)]
data

,Initial Solution,Moves
6,"[['D0', 'C61', 'D0'], ['D0', 'C24', 'C50', 'D0...","null, null, ZoneDestroyCustomer, GreedyRepairC..."
9,"[['D0', 'C61', 'D0'], ['D0', 'C24', 'C50', 'D0...","LongestWaitingTimeDestroyStation, Deterministi..."
54,"[['D0', 'C26', 'D0'], ['D0', 'C24', 'C25', 'C6...","LongestWaitingTimeDestroyStation, Deterministi..."
60,"[['D0', 'C26', 'D0'], ['D0', 'C24', 'C25', 'D0...","null, null, ZoneDestroyCustomer, GreedyRepairC..."
62,"[['D0', 'C26', 'D0'], ['D0', 'C24', 'C25', 'D0...","null, null, ZoneDestroyCustomer, GreedyRepairC..."
...,...,...
35342,"[['D0', 'C21', 'D0'], ['D0', 'C67', 'C66', 'D0...","null, null, ZoneDestroyCustomer, GreedyRepairC..."
35354,"[['D0', 'C78', 'C67', 'D0'], ['D0', 'C24', 'D0...","null, null, ZoneDestroyCustomer, GreedyRepairC..."
35368,"[['D0', 'C96', 'C67', 'C24', 'D0'], ['D0', 'C5...","null, null, ZoneDestroyCustomer, GreedyRepairC..."
35397,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',...","null, null, ZoneDestroyCustomer, GreedyRepairC..."


Questi passaggi servono ad isolare i luoghi contenuti nella colonna "Initial Solution", in modo da poter costruire una matrice a seconda della mossa pressa in considerazione 

In [5]:
init_sol = data['Initial Solution']

sol = []
for row in init_sol:
    for elem in row.split("'], "):
        sol.append(elem)
        
for i in range(len(sol)):
    sol[i] = sol[i].replace("[['","")
    sol[i] = sol[i].replace("']]","")
    sol[i] = sol[i].replace("'","")
    sol[i] = sol[i].replace(",","")
    sol[i] = sol[i].replace("[","")
    
data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
data["Initial Solution"] = data["Initial Solution"].str.replace("]","")
data["Initial Solution"] = data["Initial Solution"].str.replace(",","")
data["Initial Solution"] = data["Initial Solution"].str.replace("'","")

init_sol = data["Initial Solution"]

places = []
for row in init_sol:
    for elem in row.split():
        places.append(elem)

places = np.array(places)
distinct_places = np.unique(places)


In [6]:
# creazione dizionario  
dict = {}

# creazione dataframe per ogni indice del dizionario, dove l'indice è la lunghezza di "sol"
for index, elem in enumerate(sol):
        zeros = np.zeros([len(distinct_places), len(distinct_places)])
        dict[index] = pd.DataFrame(zeros, index=distinct_places, columns=distinct_places)

In [7]:
for index, stringa in enumerate(sol):
    
    temp = stringa
    
    split = temp.split(" ")                         
    split = np.array(split)
    
    for i in range(len(split)-1):
            dict[index].loc[split[i], split[i+1]] = 1

In [8]:
dict[0]

,C1,C10,C100,C11,C12,C13,C14,C15,C16,C17,C18,C19,C2,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C3,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C4,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C5,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C6,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C7,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C8,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C9,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,D0,S0,S1,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S2,S20,S3,S4,S5,S7,S9
C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Somma dei dataframe risultanti in un unico dataframe e salvataggio in un file .csv con lo stesso nome della mossa relativa al dataframe.

In [9]:
# inizializzo il risultato come il primo dataframe del dizionario
df_sum = dict[0]

# itero su tutti gli altri dataframe del dizionario
for i in range(1, len(dict)):
    # sommo il dataframe corrente al risultato
    df_sum = df_sum.add(dict[i])

df_sum.to_csv(user_input)
df_sum


,C1,C10,C100,C11,C12,C13,C14,C15,C16,C17,C18,C19,C2,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C3,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C4,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C5,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C6,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C7,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C8,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C9,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,D0,S0,S1,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S2,S20,S3,S4,S5,S7,S9
C1,0.0,4.0,105.0,2.0,26.0,3.0,6.0,0.0,0.0,0.0,1.0,0.0,338.0,0.0,2.0,1.0,1.0,1.0,0.0,5.0,12.0,0.0,3.0,40.0,6.0,0.0,9.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,188.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,109.0,6.0,0.0,0.0,15.0,9.0,4.0,0.0,9.0,0.0,0.0,24.0,0.0,0.0,42.0,0.0,0.0,28.0,0.0,0.0,10.0,1.0,6.0,0.0,0.0,0.0,0.0,14.0,0.0,2.0,2.0,6.0,19.0,92.0,0.0,39.0,3.0,34.0,2.0,12.0,1.0,4.0,17.0,377.0,309.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,1.0,15.0,0.0
C10,17.0,0.0,8.0,442.0,68.0,56.0,26.0,4.0,4.0,6.0,2.0,1.0,80.0,9.0,8.0,19.0,13.0,0.0,20.0,10.0,16.0,8.0,53.0,0.0,17.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,2.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,3.0,0.0,7.0,0.0,0.0,3.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,56.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,26.0,105.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,55.0,648.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,10.0,0.0,0.0,5.0,0.0,1.0,38.0,3.0
C100,28.0,6.0,0.0,6.0,46.0,0.0,1.0,0.0,22.0,0.0,0.0,5.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,7.0,0.0,0.0,23.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,11.0,41.0,4.0,1.0,3.0,0.0,0.0,0.0,58.0,21.0,0.0,0.0,0.0,19.0,100.0,20.0,146.0,0.0,192.0,3.0,50.0,72.0,0.0,12.0,14.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,858.0,67.0,16.0,7.0,0.0
C11,8.0,82.0,2.0,0.0,119.0,132.0,33.0,2.0,8.0,18.0,11.0,5.0,0.0,6.0,1.0,0.0,7.0,1.0,39.0,10.0,0.0,18.0,3.0,4.0,17.0,0.0,12.0,3.0,0.0,9.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,286.0,4.0,6.0,16.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,960.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,4.0,1.0,27.0,6.0
C12,0.0,20.0,6.0,37.0,0.0,56.0,540.0,4.0,264.0,127.0,0.0,143.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,39.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,46.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,17.0,0.0,4.0,150.0,0.0,0.0,0.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,7.0,6.0,5.0,709.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S3,16.0,9.0,11.0,4.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,51.0,16.0,20.0,7.0,4.0,3.0,0.0,20.0,4.0,0.0,0.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,5.0,0.0,0.0,7.0,0.0,0.0,0.0,18.0,0.0,14.0,0.0,2.0,106.0,104.0,0.0,34.0,18.0,1.0,3.0,5.0,1.0,3.0,3.0,18.0,5.0,2.0,6.0,10.0,1.0,0.0,0.0,10.0,35.0,25.0,28.0,3.0,2768.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,14.0,0.0,79.0,36.0,4.0,0.0,1.0,0

Conversione dei dataframe in matrici numpy e somma di tutte le matrici in un'unica matrice risultante.

In [ ]:
for i in range(len(dict)):
    dict[i] = dict[i].to_numpy()

np_sum = df_sum.to_numpy()
print(np_sum)

Da eliminare 

In [ ]:
for i in range(len(dict)):
    dict[i] = dict[i].to_numpy()

# Inizializzo una variabile per memorizzare il risultato
result = None

# Creo un ciclo che iteri sulla lista di matrici
for matrix in dict.values():
    # Se è il primo ciclo, il risultato è uguale alla prima matrice
    if result is None:
        result = matrix
    # Altrimenti, sommo il risultato con la matrice corrente
    else:
        result = np.add(result, matrix)

# Stampo il risultato
print(result)